# Ejemplo 5

Calculo de la densidad marginal en n-dimensiones

Para el caso de funciones de densidad discretas

$$ P_x(x) = P(X=x) = \sum_{y \in \Omega_y}^{} P_{XY} (x, y) $$

$$ P_y(y) = P(Y=y) = \sum_{x \in \Omega_x}^{} P_{XY} (x, y) $$

Para el caso de funciones de densidad continuas

$$ f_x(x) = P(x \in B) = \int_{-\infty }^{\infty} f_{XY} (x, y) \,dx $$

$$ f_y(y) = P(y \in B) = \int_{-\infty }^{\infty} f_{XY} (x, y) \,dx $$

Para n-dimensiones

$$ f_{x_i}(x_i) = P(x_i \in B) = \int_{-\infty }^{\infty} \dots \int_{-\infty }^{\infty} f(x_i, \dots,  x_n) \,dx_i \dots dx_n $$

# Caso discreto

(Ejercicio obtenido del libro> Devore, J. L. (2009). Probabilidad y estadística para ingeniería y ciencias. Cengage Learning Editores, pág. 100)

Suponiendo un problema de la siguieren forma:

$$
f(x, y) =
\begin{cases}
\frac{x (1+3y)^2}{4} & \text{si } 0 < x < 2, 0 < y < 1 \\
0 & \text{ en otro caso}
\end{cases}
$$

Para el caso discreto


Definiendo las variables a utilizar y la función.

In [ ]:
import sympy as sp

x, y = sp.symbols('x y')

x_values = [1, 2]
y_values = [0, 1]

f_xy = (x * (1 + 3 * y)**2) / 4

Calculo de las densidades marginales de x, y

$$ P_x(x) = P(X=x) = \sum_{y \in \Omega_y}^{} P_{XY} (x, y) $$

$$ P_y(y) = P(Y=y) = \sum_{x \in \Omega_x}^{} P_{XY} (x, y) $$

In [ ]:

f_X = {xi: sum(f_xy.subs(x, xi).subs(y, yi) for yi in y_values) for xi in x_values}

f_Y = {yi: sum(f_xy.subs(x, xi).subs(y, yi) for xi in x_values) for yi in y_values}


Resultados

In [ ]:
print("Masa marginal de X, f_X(x):")
for xi, val in f_X.items():
    print(f"f_X({xi}) = {val}")

print("\nMasa marginal de Y, f_Y(y):")
for yi, val in f_Y.items():
    print(f"f_Y({yi}) = {val}")

Masa marginal de X, f_X(x):
f_X(1) = 17/4
f_X(2) = 17/2

Masa marginal de Y, f_Y(y):
f_Y(0) = 3/4
f_Y(1) = 12


# Caso continuo

Utilizando el mismo ejemplo para el caso continuo:

$$
f(x, y) =
\begin{cases}
\frac{x (1+3y)^2}{4} & \text{si } 0 < x < 2, 0 < y < 1 \\
0 & \text{ en otro caso}
\end{cases}
$$

Definiendo las variables a utilizar y de la función de densidad conjunta.

In [ ]:
import sympy as sp

x, y = sp.symbols('x y')

f_xy = (x * (1 + 3 * y)**2) / 4

Calculo de las densidades marginales de x, y

$$ f_x(x) = P(x \in B) = \int_{-\infty }^{\infty} f_{XY} (x, y) \,dy $$

$$ f_y(y) = P(y \in B) = \int_{-\infty }^{\infty} f_{XY} (x, y) \,dx $$

In [ ]:
f_X = sp.integrate(f_xy, (y, 0, 1))

f_Y = sp.integrate(f_xy, (x, 0, 2))


Resultados

In [ ]:
print("Densidad marginal de X, f_X(x):")
sp.pprint(f_X)

print("\nDensidad marginal de Y, f_Y(y):")
sp.pprint(f_Y)

Densidad marginal de X, f_X(x):
7⋅x
───
 4 

Densidad marginal de Y, f_Y(y):
   2          
9⋅y          1
──── + 3⋅y + ─
 2           2


# Para n-dimensiones

In [ ]:
import sympy as sp
import numpy as np

def get_function_and_limits():
    num_vars = int(input("Ingrese el número de variables aleatorias (dimensiones): "))

    variables = sp.symbols(' '.join([f'x{i+1}' for i in range(num_vars)]))  # Crear variables simbólicas
    function_str = input("Ingrese la función de probabilidad conjunta f(x1, x2, ..., xn): ")
    f = sp.sympify(function_str)

    limits = {}
    choice = input("¿Es el caso continuo o discreto? (Ingrese 'c' para continuo o 'd' para discreto): ").lower()

    if choice == 'c':
        for var in variables:
            lower_limit = float(input(f"Ingrese el límite inferior de integración para {var}: "))
            upper_limit = float(input(f"Ingrese el límite superior de integración para {var}: "))
            limits[var] = (lower_limit, upper_limit)
    elif choice == 'd':
        for var in variables:
            values = input(f"Ingrese los valores discretos para {var} separados por comas: ")
            limits[var] = [float(v) for v in values.split(',')]

    return f, limits, choice

def calculate_marginal_densities(f, limits, choice):
    variables = list(limits.keys())
    marginal_densities = {}

    for var in variables:
        if choice == 'c':
            marginal_density = f
            for other_var in variables:
                if other_var != var:
                    lower, upper = limits[other_var]
                    marginal_density = sp.integrate(marginal_density, (other_var, lower, upper))
        elif choice == 'd':
            marginal_density = {}
            values = limits[var]
            other_vars = {k: v for k, v in limits.items() if k != var}

            # Recorremos todos los valores posibles de las otras variables
            other_values = np.array(np.meshgrid(*other_vars.values())).T.reshape(-1, len(other_vars))

            for value in values:
                marginal_sum = 0
                for vals in other_values:
                    product = value
                    for i, other_var in enumerate(other_vars.keys()):
                        product *= vals[i]
                    marginal_sum += product
                marginal_density[value] = marginal_sum

            # Normalizar la densidad marginal
            total_sum = sum(marginal_density.values())
            if total_sum != 0:
                for value in marginal_density:
                    marginal_density[value] /= total_sum

        marginal_densities[var] = marginal_density

    return marginal_densities

# Ejemplo de uso

# Descripción del Problema

Dado una función de probabilidad conjunta $$ f(x_1, x_2, \ldots, x_n) $$ y los límites de integración (en el caso continuo) o los valores discretos (en el caso discreto) para cada variable, el programa calcula las densidades marginales para cada variable.

# Función Conjunta

Consideremos una función conjunta para $$ n = 5 $$ dimensiones como ejemplo.

## Caso Continuo

La función de probabilidad conjunta es:

$$
f(x_1, x_2, x_3, x_4, x_5) = \frac{x_1 x_2 x_3 x_4 x_5}{120}
$$

con límites de integración:

$$
0 \leq x_i \leq 1 \text{ para } i = 1, 2, 3, 4, 5
$$

## Caso Discreto

Para valores discretos, consideramos:

$$
f(x_1, x_2, x_3, x_4, x_5) = \frac{x_1 x_2 x_3 x_4 x_5}{120}
$$

con los siguientes valores para cada variable:

$$( X_1 ): \{0.1, 0.5, 1\}$$
$$( X_2 ): \{0.2, 0.4, 1\}$$
$$( X_3 ): \{0.3, 0.6, 1\}$$
$$( X_4 ): \{0.4, 0.8, 1\}$$
$$( X_5 ): \{0.5, 1\}


In [ ]:
f_c = "x1*x2*x3*x4*x5/120"
f_c = sp.symbols(f_c)
var = sp.symbols(' '.join([f'x{i+1}' for i in range(5)]))
limits_c = {var[0]: (0, 1), var[1]: (0, 1), var[2]: (0, 1), var[3]: (0, 1), var[4]: (0, 1)}
choice_c = 'c'

f_d = "x1*x2*x3*x4*x5/120"
f_d = sp.symbols(f_d)
var = sp.symbols(' '.join([f'x{i+1}' for i in range(5)]))
limits_d = {var[0]: [0, 0.5, 1], var[1]: [0.2, 0.4, 1], var[2]: [0.3, 0.6, 1], var[3]: [0.4, 0.8, 1], var[4]: [0.5, 1]}
choice_d = 'd'

def main():
    op = 0

    # Descomentar las siguientes 2 linea para ingresar los datos manualmente
    # op = 1
    # f, limits, choice = get_function_and_limits()


    if op == 1:
        marginal_densities = calculate_marginal_densities(f, limits, choice)
        print("\nLas densidades marginales calculadas son:")
        for var, density in marginal_densities.items():
            print(f"f_{var}({var}) = {density}")
    else:
        marginal_densities_c = calculate_marginal_densities(f_c, limits_c, choice_c)
        marginal_densities_d = calculate_marginal_densities(f_d, limits_d, choice_d)

        print("\nLas densidades marginales calculadas son:")
        for var, density in marginal_densities_c.items():
            print(f"f_{var}({var}) = {density}")

        print("\nLas densidades marginales calculadas son:")
        for var, density in marginal_densities_d.items():
            print(f"f_{var}({var}) = {density}")

if __name__ == "__main__":
    main()


Las densidades marginales calculadas son:
f_x1(x1) = x1*x2*x3*x4*x5/120
f_x2(x2) = x1*x2*x3*x4*x5/120
f_x3(x3) = x1*x2*x3*x4*x5/120
f_x4(x4) = x1*x2*x3*x4*x5/120
f_x5(x5) = x1*x2*x3*x4*x5/120

Las densidades marginales calculadas son:
f_x1(x1) = {0: 0.0, 0.5: 0.33333333333333337, 1: 0.6666666666666667}
f_x2(x2) = {0.2: 0.12499999999999999, 0.4: 0.24999999999999997, 1: 0.625}
f_x3(x3) = {0.3: 0.15789473684210525, 0.6: 0.3157894736842105, 1: 0.5263157894736843}
f_x4(x4) = {0.4: 0.18181818181818185, 0.8: 0.3636363636363637, 1: 0.4545454545454545}
f_x5(x5) = {0.5: 0.3333333333333333, 1: 0.6666666666666666}
